``Panel`` provides functionality to leverage parameters and dependencies between parameters as expressed by ``param`` in a simple way to encapsulate dashboards as declarative and self-contained classes.

Parameters are Python attributes extended using the [Param library](https://github.com/ioam/param) to support types, ranges, and documentation, which turns out to be just the information you need to automatically create widgets for each parameter. 

# Parameters and widgets

To use the functionality first declare some Parameterized classes with various Parameters:

In [ ]:
import param
import datetime as dt

class BaseClass(param.Parameterized):
    x                       = param.Parameter(default=3.14,doc="X position")
    y                       = param.Parameter(default="Not editable",constant=True)
    string_value            = param.String(default="str",doc="A string")
    num_int                 = param.Integer(50000,bounds=(-200,100000))
    unbounded_int           = param.Integer(23)
    float_with_hard_bounds  = param.Number(8.2,bounds=(7.5,10))
    float_with_soft_bounds  = param.Number(0.5,bounds=(0,None),softbounds=(0,2))
    unbounded_float         = param.Number(30.01,precedence=0)
    hidden_parameter        = param.Number(2.718,precedence=-1)
    integer_range           = param.Range(default=(3,7),bounds=(0, 10))
    float_range             = param.Range(default=(0,1.57),bounds=(0, 3.145))
    dictionary              = param.Dict(default={"a":2, "b":9})
    
class Example(BaseClass):
    """An example Parameterized class"""
    timestamps = []

    boolean                 = param.Boolean(True, doc="A sample Boolean parameter")
    color                   = param.Color(default='#FFFFFF')
    date                    = param.Date(dt.datetime(2017, 1, 1),
                                         bounds=(dt.datetime(2017, 1, 1), dt.datetime(2017, 2, 1)))
    select_string           = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
    select_fn               = param.ObjectSelector(default=list,objects=[list,set,dict])
    int_list                = param.ListSelector(default=[3,5], objects=[1,3,5,7,9],precedence=0.5)
    single_file             = param.FileSelector(path='../../*/*.py*',precedence=0.5)
    multiple_files          = param.MultiFileSelector(path='../../*/*.py?',precedence=0.5)
    record_timestamp        = param.Action(lambda x: x.timestamps.append(dt.datetime.now()), 
                                           doc="""Record timestamp.""",precedence=0.7)
    
Example.num_int

As you can see, declaring Parameters depends only on the separate Param library.  Parameters are a simple idea with some properties that are crucial for helping you create clean, usable code:

- The Param library is pure Python with no dependencies, which makes it easy to include in any code without tying it to a particular GUI or widgets library, or even to the Jupyter notebook.  
- Parameter declarations focus on semantic information relevant to your domain, allowing you to avoid polluting your domain-specific code with anything that ties it to a particular way of displaying or interacting with it. 
- Parameters can be defined wherever they make sense in your inheritance hierarchy, allowing you to document, type, and range-limit them once, with all of those properties inherited by any base class.  E.g. parameters work the same here whether they were declared in `BaseClass` or `Example`, which makes it easy to provide this metadata once, and avoiding duplicating it throughout the code wherever ranges or types need checking or documentation needs to be stored.

If you then decide to use these Parameterized classes in a notebook environment, you can import `panel` and easily display and edit the parameter values as an optional additional step:

In [ ]:
import panel as pn

pn.extension()

base = BaseClass()
pn.Row(Example, base)

As you can see, `Panel` does not need to be provided with any knowledge of your domain-specific application, not even the names of your parameters; it simply displays widgets for whatever Parameters may have been defined on that object.  Using Param with ``Panel`` thus achieves a nearly complete separation between your domain-specific code and your display code, making it vastly easier to maintain both of them over time.  Here even the `msg` button behavior was specified declaratively, as an action that can be invoked (printing \"Hello\") independently of whether it is used in a GUI or other context.

Interacting with the widgets above is only supported in the notebook and on bokeh server, but you can also export static renderings of the widgets to a file or web page.  

By default, editing values in this way requires you to run the notebook cell by cell -- when you get to the above cell, edit the values as you like, and then move on to execute subsequent cells, where any reference to those parameter values will use your interactively selected setting:

In [ ]:
Example.unbounded_int

In [ ]:
Example.num_int

The reverse is possible, editing a parameter from Python will automatically update any widgets that were generated from the parameter:

In [ ]:
Example.int_list = [1, 7]

Example.timestamps records the times you pressed the "record timestamp" button.

In [ ]:
Example.timestamps

As you can see, you can access the parameter values at the class level from within the notebook to control behavior explicitly, e.g. to select what to show in subsequent cells.  Moreover, any instances of the Parameterized classes in your own code will now use the new parameter values unless specifically overridden in that instance, so you can now import and use your domain-specific library however you like, knowing that it will use your interactive selections wherever those classes appear.  None of the domain-specific code needs to know or care that you used ``Panel``; it will simply see new values for whatever attributes were changed interactively.  ``Panel`` thus allows you to provide notebook-specific, domain-specific interactive functionality without ever tying your domain-specific code to the notebook environment. However that is only the beginning!"

# Parameter dependencies
Declaring parameters is usually only the beginning of a workflow, in most applications these parameters are then tied to some computation. To express the relationship between a computation and the parameters it depends on the ``param.depends`` decorator may be used on Parameterized methods. This provides a hint to ``panel`` and other libraries (e.g. HoloViews) that the method should be recomputed when a parameter changes.

As a straightforward example without any additional dependencies we will write a small class which returns an ASCII representation of a sine wave, which depends on `phase` and `frequency` parameters. If we supply the ``.view`` method to a ``Panel`` layout it will automatically recompute and update the view when one or more of the parameters change:

In [ ]:
import numpy as np

class Sine(param.Parameterized):

    phase = param.Number(default=0, bounds=(0, np.pi))

    frequency = param.Number(default=1, bounds=(0.1, 2))

    @param.depends('phase', 'frequency')
    def view(self):
        y = np.sin(np.linspace(0, np.pi*3, 40)*self.frequency+self.phase)
        y = ((y-y.min())/y.ptp())*20
        array = np.array([list((' '*(int(round(d))-1) + '*').ljust(20)) for d in y])
        return pn.pane.Str('\n'.join([''.join(r) for r in array.T]), height=325, width=500)

sine = Sine(name='ASCII Sine Wave')
pn.Row(sine.param, sine.view)

The parameterized and annotated ``view`` method could return any one of the types handled by the [Pane objects](./Panes.ipynb) panel provides, making it easy to link parameters and their associated widgets to a plot or other output. Parameterized classes can therefore be a very useful pattern for encapsulating a part of a computational workflow with an associated visualization, declaratively expressing the dependencies between the parameters and the computation.

Another common pattern is linking the values of one parameter to another parameter, e.g. when dependencies between parameters exist. In the example below we will define two parameters one for the continent and one for the country. Since we want the selection of valid countries to change when we change the continent we define a method to do that for us. In order to link the two we express the dependency using the ``param.depends`` decorator and then ensure that we want to run the method whenever the continent changes by setting ``watch=True``.

Additionally we define a ``view`` method which returns an HTML iframe displaying the country in a Google Map.

In [ ]:
class GoogleMapViewer(param.Parameterized):
    
    continent = param.ObjectSelector(default='Asia', objects=['Africa', 'Asia', 'Europe'])
    
    country = param.ObjectSelector(default='China', objects=['China', 'Thailand', 'Japan'])
    
    _countries = {'Africa': ['Ghana', 'Togo', 'South Africa', 'Tanzania'],
                  'Asia'  : ['China', 'Thailand', 'Japan'],
                  'Europe': ['Austria', 'Bulgaria', 'Greece', 'Portugal', 'Switzerland']}
    
    @param.depends('continent', watch=True)
    def _update_countries(self):
        countries = self._countries[self.continent]
        self.params('country').objects = countries
        self.country = countries[0]

    @param.depends('country')
    def view(self):
        iframe = """
        <iframe width="800" height="400" src="http://maps.google.com/maps?q={country}&z=6&output=embed"
        frameborder="0" scrolling="no" marginheight="0" marginwidth="0"></iframe>
        """.format(country=self.country)
        return pn.pane.HTML(iframe, height=400)
        
viewer = GoogleMapViewer(name='Google Map Viewer')
pn.Row(viewer.param, viewer.view)

Whenever the continent changes it will now eagerly execute the ``_update_countries`` method changing the list of countries that is displayed, which in turn triggers an update in the view method updating the map.

In this user guide we have seen how to leverage ``param`` to declare parameters, which ``panel`` can turn into a GUI with no overhead. Additionally we have seen how to link parameters to views and other parameters using the ``param.depends`` operator. This approach allows building complex and reactive ``panels``. In the [next user guide](Pipelines.ipynb) we will discover how to link multiple such classes into pipelines, making it possible to encapsulate complex workflows in clean self-contained classes.